# Import packages & functions

In [1]:
!module load fsl/6.0.6.2 

In [2]:
import os
import sys
import json
import argparse
import numpy as np
import math
from einops import rearrange
import time
import random
import string
import h5py
from tqdm import tqdm
import webdataset as wds
import gc
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torchvision import transforms
from accelerate import Accelerator, DeepSpeedPlugin
import pandas as pd
# SDXL unCLIP requires code from https://github.com/Stability-AI/generative-models/tree/main
sys.path.append('generative_models/')
import sgm
from generative_models.sgm.modules.encoders.modules import FrozenOpenCLIPImageEmbedder # bigG embedder

# tf32 data type is faster than standard float32
torch.backends.cuda.matmul.allow_tf32 = True

# custom functions #
import utils

In [3]:
my_env = os.environ.copy()
my_env["PATH"] = "/home/rk1593/.conda/envs/rt_mindEye2/bin:" + my_env["PATH"]
os.environ.update(my_env)

In [4]:
### Multi-GPU config ###
local_rank = os.getenv('RANK')
if local_rank is None: 
    local_rank = 0
else:
    local_rank = int(local_rank)
print("LOCAL RANK ", local_rank)  

data_type = torch.float16 # change depending on your mixed_precision
num_devices = torch.cuda.device_count()
if num_devices==0: num_devices = 1

# First use "accelerate config" in terminal and setup using deepspeed stage 2 with CPU offloading!
accelerator = Accelerator(split_batches=False, mixed_precision="fp16")
if utils.is_interactive(): # set batch size here if using interactive notebook instead of submitting job
    global_batch_size = batch_size = 8
else:
    global_batch_size = os.environ["GLOBAL_BATCH_SIZE"]
    batch_size = int(os.environ["GLOBAL_BATCH_SIZE"]) // num_devices

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


LOCAL RANK  0


In [5]:
print("PID of this process =",os.getpid())
device = accelerator.device
print("device:",device)
world_size = accelerator.state.num_processes
distributed = not accelerator.state.distributed_type == 'NO'
num_devices = torch.cuda.device_count()
if num_devices==0 or not distributed: num_devices = 1
num_workers = num_devices
print(accelerator.state)

print("distributed =",distributed, "num_devices =", num_devices, "local rank =", local_rank, "world size =", world_size, "data_type =", data_type)
print = accelerator.print # only print if local_rank=0

PID of this process = 485757
device: cuda
Distributed environment: DistributedType.NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: fp16

distributed = False num_devices = 1 local rank = 0 world size = 1 data_type = torch.float16


# Configurations

In [6]:
# if running this interactively, can specify jupyter_args here for argparser to use
if utils.is_interactive():
    model_name = "testing"
    print("model_name:", model_name)
    
    # global_batch_size and batch_size should already be defined in the above cells
    # other variables can be specified in the following string:
    jupyter_args = f"--data_path=/scratch/gpfs/rk1593/rt_mindEye \
                    --cache_dir=/scratch/gpfs/rk1593/rt_mindEye/cache \
                    --model_name=multisubject_subj01_1024hid_nolow_300ep \
                    --no-multi_subject --no-visualize_prior --subj=1 --batch_size=64 --num_sessions=1 \
                    --hidden_dim=1024 --clip_scale=1. \
                    --no-blurry_recon --blur_scale=.5  \
                    --seq_past=0 --seq_future=0 \
                    --use_prior --prior_scale=30 --multisubject_ckpt=/scratch/gpfs/rk1593/rt_mindEye/multisubject_subj01_1024hid_nolow_300ep \
                    --n_blocks=4 --max_lr=3e-4 --mixup_pct=.33 --num_epochs=300 --no-use_image_aug \
                    --ckpt_interval=50 --ckpt_saving --wandb_log"# \
                    #--multisubject_ckpt=../train_logs/multisubject_subj01_acc"


        
        
    print(jupyter_args)
    jupyter_args = jupyter_args.split()
    
    from IPython.display import clear_output # function to clear print outputs in cell
    %load_ext autoreload 
    # this allows you to change functions in models.py or utils.py and have this notebook automatically update with your revisions
    %autoreload 2 

model_name: testing
--data_path=/scratch/gpfs/rk1593/rt_mindEye                     --cache_dir=/scratch/gpfs/rk1593/rt_mindEye/cache                     --model_name=multisubject_subj01_1024hid_nolow_300ep                     --no-multi_subject --no-visualize_prior --subj=1 --batch_size=64 --num_sessions=1                     --hidden_dim=1024 --clip_scale=1.                     --no-blurry_recon --blur_scale=.5                      --seq_past=0 --seq_future=0                     --use_prior --prior_scale=30 --multisubject_ckpt=/scratch/gpfs/rk1593/rt_mindEye/multisubject_subj01_1024hid_nolow_300ep                     --n_blocks=4 --max_lr=3e-4 --mixup_pct=.33 --num_epochs=300 --no-use_image_aug                     --ckpt_interval=50 --ckpt_saving --wandb_log


In [7]:
parser = argparse.ArgumentParser(description="Model Training Configuration")
parser.add_argument(
    "--model_name", type=str, default="testing",
    help="name of model, used for ckpt saving and wandb logging (if enabled)",
)
parser.add_argument(
    "--data_path", type=str, default=os.getcwd(),
    help="Path to where NSD data is stored / where to download it to",
)
parser.add_argument(
    "--cache_dir", type=str, default=os.getcwd(),
    help="Path to where misc. files downloaded from huggingface are stored. Defaults to current src directory.",
)
parser.add_argument(
    "--subj",type=int, default=1, choices=[1,2,3,4,5,6,7,8],
    help="Validate on which subject?",
)
parser.add_argument(
    "--multisubject_ckpt", type=str, default=None,
    help="Path to pre-trained multisubject model to finetune a single subject from. multisubject must be False.",
)
parser.add_argument(
    "--num_sessions", type=int, default=0,
    help="Number of training sessions to include (zero = all sessions)",
)
parser.add_argument(
    "--use_prior",action=argparse.BooleanOptionalAction,default=True,
    help="whether to train diffusion prior (True) or just rely on retrieval part of the pipeline (False)",
)
parser.add_argument(
    "--visualize_prior",action=argparse.BooleanOptionalAction,default=False,
    help="output visualizations from unCLIP every ckpt_interval (requires more memory!)",
)
parser.add_argument(
    "--batch_size", type=int, default=16,
    help="Batch size can be increased by 10x if only training retreival submodule and not diffusion prior",
)
parser.add_argument(
    "--wandb_log",action=argparse.BooleanOptionalAction,default=False,
    help="whether to log to wandb",
)
parser.add_argument(
    "--resume_from_ckpt",action=argparse.BooleanOptionalAction,default=False,
    help="if not using wandb and want to resume from a ckpt",
)
parser.add_argument(
    "--wandb_project",type=str,default="stability",
    help="wandb project name",
)
parser.add_argument(
    "--mixup_pct",type=float,default=.33,
    help="proportion of way through training when to switch from BiMixCo to SoftCLIP",
)
parser.add_argument(
    "--blurry_recon",action=argparse.BooleanOptionalAction,default=True,
    help="whether to output blurry reconstructions",
)
parser.add_argument(
    "--blur_scale",type=float,default=.5,
    help="multiply loss from blurry recons by this number",
)
parser.add_argument(
    "--clip_scale",type=float,default=1.,
    help="multiply contrastive loss by this number",
)
parser.add_argument(
    "--prior_scale",type=float,default=30,
    help="multiply diffusion prior loss by this",
)
parser.add_argument(
    "--use_image_aug",action=argparse.BooleanOptionalAction,default=False,
    help="whether to use image augmentation",
)
parser.add_argument(
    "--num_epochs",type=int,default=150,
    help="number of epochs of training",
)
parser.add_argument(
    "--multi_subject",action=argparse.BooleanOptionalAction,default=False,
)
parser.add_argument(
    "--new_test",action=argparse.BooleanOptionalAction,default=True,
)
parser.add_argument(
    "--n_blocks",type=int,default=4,
)
parser.add_argument(
    "--hidden_dim",type=int,default=1024,
)
parser.add_argument(
    "--seq_past",type=int,default=0,
)
parser.add_argument(
    "--seq_future",type=int,default=0,
)
parser.add_argument(
    "--lr_scheduler_type",type=str,default='cycle',choices=['cycle','linear'],
)
parser.add_argument(
    "--ckpt_saving",action=argparse.BooleanOptionalAction,default=True,
)
parser.add_argument(
    "--ckpt_interval",type=int,default=5,
    help="save backup ckpt and reconstruct every x epochs",
)
parser.add_argument(
    "--seed",type=int,default=42,
)
parser.add_argument(
    "--max_lr",type=float,default=3e-4,
)

if utils.is_interactive():
    args = parser.parse_args(jupyter_args)
else:
    args = parser.parse_args()

# create global variables without the args prefix
for attribute_name in vars(args).keys():
    globals()[attribute_name] = getattr(args, attribute_name)
    
# seed all random functions
utils.seed_everything(seed)

outdir = os.path.abspath(f'/scratch/gpfs/rk1593/rt_mindEye/train_logs/{model_name}')
if not os.path.exists(outdir) and ckpt_saving:
    os.makedirs(outdir,exist_ok=True)
    
if use_image_aug or blurry_recon:
    import kornia
    from kornia.augmentation.container import AugmentationSequential
if use_image_aug:
    img_augment = AugmentationSequential(
        kornia.augmentation.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1, p=0.3),
        same_on_batch=False,
        data_keys=["input"],
    )
    
if multi_subject:
    subj_list = np.arange(1,9)
    subj_list = subj_list[subj_list != subj]
else:
    subj_list = [subj]

print("subj_list", subj_list, "num_sessions", num_sessions)

subj_list [1] num_sessions 1


# Prep data, models, and dataloaders

### Creating wds dataloader, preload betas and all 73k possible images

In [8]:
def my_split_by_node(urls): return urls
num_voxels_list = []

if multi_subject:
    nsessions_allsubj=np.array([40, 40, 32, 30, 40, 32, 40, 30])
    num_samples_per_epoch = (750*40) // num_devices 
else:
    num_samples_per_epoch = (750*num_sessions) // num_devices 

print("dividing batch size by subj_list, which will then be concatenated across subj during training...") 
batch_size = batch_size // len(subj_list)

num_iterations_per_epoch = num_samples_per_epoch // (batch_size*len(subj_list))

print("batch_size =", batch_size, "num_iterations_per_epoch =",num_iterations_per_epoch, "num_samples_per_epoch =",num_samples_per_epoch)

dividing batch size by subj_list, which will then be concatenated across subj during training...
batch_size = 64 num_iterations_per_epoch = 11 num_samples_per_epoch = 750


In [9]:
train_data = {}
train_dl = {}
num_voxels = {}
voxels = {}
for s in subj_list:
    print(f"Training with {num_sessions} sessions")
    if multi_subject:
        train_url = f"{data_path}/wds/subj0{s}/train/0.tar"
    else:
        train_url = f"{data_path}/wds/subj0{s}/train/0.tar"
    print(train_url)

    train_data[f'subj0{s}'] = wds.WebDataset(train_url,resampled=True,nodesplitter=my_split_by_node)\
                        .shuffle(750, initial=1500, rng=random.Random(42))\
                        .decode("torch")\
                        .rename(behav="behav.npy", past_behav="past_behav.npy", future_behav="future_behav.npy", olds_behav="olds_behav.npy")\
                        .to_tuple(*["behav", "past_behav", "future_behav", "olds_behav"])
    train_dl[f'subj0{s}'] = torch.utils.data.DataLoader(train_data[f'subj0{s}'], batch_size=batch_size, shuffle=False, drop_last=True, pin_memory=True)

    f = h5py.File(f'{data_path}/betas_all_subj0{s}_fp32_renorm.hdf5', 'r')
    betas = f['betas'][:]
    print("np.shape(betas): ", np.shape(betas))
    betas = torch.Tensor(betas).to("cpu").to(data_type)
    num_voxels_list.append(betas[0].shape[-1])
    num_voxels[f'subj0{s}'] = betas[0].shape[-1]
    voxels[f'subj0{s}'] = betas
    print(f"num_voxels for subj0{s}: {num_voxels[f'subj0{s}']}")

print("Loaded all subj train dls and betas!\n")


seq_len = seq_past + 1 + seq_future
print(f"currently using {seq_len} seq_len (chose {seq_past} past behav and {seq_future} future behav)")

Training with 1 sessions
/scratch/gpfs/rk1593/rt_mindEye/wds/subj01/train/0.tar
np.shape(betas):  (30000, 15724)
num_voxels for subj01: 15724
Loaded all subj train dls and betas!

currently using 1 seq_len (chose 0 past behav and 0 future behav)


In [10]:
betas_custom = np.zeros(shape=(30000, 15724))
import os
import nibabel as nib
from subprocess import call
from datetime import datetime, date


In [11]:
mask_img = nib.load('/scratch/gpfs/rk1593/rt_mindEye/raw/subj01/nsdgeneral.nii.gz')
mask_np = mask_img.get_fdata()
mask_np.shape
#np.unique(mask_np, return_counts = True)

(81, 104, 83)

In [12]:
# create ref vol
# from nilearn.image import get_data, index_img, concat_imgs, new_img_like

# ndscore_bold_img = nib.load(f'/scratch/gpfs/rk1593/rt_mindEye/raw/subj01/raw/0/sub-01_ses-nsd01_task-nsdcore_run-01_bold.nii.gz')
# refvol = ndscore_bold_img.get_fdata()[:,:,:,:10]
# refvol = np.mean(refvol,axis = 3)
# refvol = new_img_like(ref_niimg = ndscore_bold_img, data = refvol,copy_header=True)
# nib.save(refvol,'/scratch/gpfs/rk1593/rt_mindEye/preprocessing_intermediates/refvol.nii')

we need fsl, talk to della people

tr 1 is 0 to 1.6

run mcflirt on whole brain so .par will give you the tr by 6 and that will be the confounds


In [ ]:
betas_custom = np.zeros(shape=(30000, 15724))
import os
import nibabel as nib
from subprocess import call
from datetime import datetime, date
from nilearn.glm.first_level import *
from nilearn import image, plotting
import nibabel as nib

from subprocess import call
from pathlib import Path
from datetime import datetime, date
from nilearn.image import get_data, index_img, concat_imgs, new_img_like
from nilearn.signal import clean
range_runs = (1,13)
# load in the bolds imgs time series and events tsv from nsd rawdata
ndscore_bold_imgs = [nib.load(f'/scratch/gpfs/rk1593/rt_mindEye/raw/subj01/raw/0/sub-01_ses-nsd01_task-nsdcore_run-{run:02d}_bold.nii.gz') for run in range(range_runs[0], range_runs[1])]
ndscore_events = [pd.read_csv(f'/scratch/gpfs/rk1593/rt_mindEye/raw/subj01/raw/0/sub-01_ses-nsd01_task-nsdcore_run-{run:02d}_events.tsv', sep = "\t", header = 0) for run in range(range_runs[0], range_runs[1])]
# create a new list of events_df's which will have the trial_type modified to be unique identifiers
new_ndscore_events = []
mask_img = nib.load('/scratch/gpfs/rk1593/rt_mindEye/masking_operation/thresholded_masks/day1_subj1/mask_transformed_0.4444440000_day1_subj1.nii.gz')
mask_on = True
if not mask_on:
    mask_img = None
lsa_glm = FirstLevelModel(t_r=1.6,slice_time_ref=.5,hrf_model='glover',
                        drift_model=None,high_pass=None,mask_img=mask_img,
                        signal_scaling=False,smoothing_fwhm=None,noise_model='ar1',
                        n_jobs=-1,verbose=-1,memory_level=1,minimize_memory=True)
from datetime import date

today = date.today()

# Month abbreviation, day and year	
d4 = today.strftime("%b-%d-%Y")
print("d4 =", d4)

def motion_correct(in_=None, reffile=None, out=None):
    # Motion correct to this run's functional reference
    # takes short 10 volume bold run and temporal avg of that as single 3d reference volume
    # create pseudo
    
    command = f"mcflirt -in {in_} -reffile {reffile} -plots -out {out}"
    A = datetime.now().timestamp(); call(command,shell=True); B = datetime.now().timestamp()
    pad_print(f"Motion correction time: {B-A:.4f}")
!module load fsl/6.0.6.2 

# this for loop is going to give us beta maps with their trial names
# we have a beta map for each stimulus trial given in all 12 runs of day 1 here
beta_maps_list = []
all_trial_names_list = []
starting_run = 0
if starting_run > 0:
    if mask_on:
        beta_maps_list = np.load("/scratch/gpfs/rk1593/rt_mindEye/masked_betas_subj1_day1_up_to_run" + \
                                 str(starting_run - 1) + "_" + d4 + ".npy").tolist()
        all_trial_names_list = np.load("/scratch/gpfs/rk1593/rt_mindEye/masked_trial_ids_up_to_run" + \
                                       str(starting_run - 1) + "_" + d4 + ".npy").tolist()

    else:
        beta_maps_list = np.load("/scratch/gpfs/rk1593/rt_mindEye/betas_subj1_day1_up_to_run" + \
                                 str(starting_run - 1) + "_" + d4 + ".npy").tolist()
        all_trial_names_list = np.load("/scratch/gpfs/rk1593/rt_mindEye/trial_ids_up_to_run" + \
                                       str(starting_run - 1) + "_" + d4 + ".npy").tolist()


for run_num,events_df in enumerate(ndscore_events):
    print(run_num)
    if run_num < starting_run:
        continue
    new_events_df = events_df
    for i_trial, trial in new_events_df.iterrows():
        new_events_df.loc[i_trial, "trial_type"] = "COCOimage_" + str(trial["73k_id"]) + "_trialNum_" + str(trial["trial_number"]) + "_runNum" + str(run_num)
    mc_params = []
    imgs = []
    raw = ndscore_bold_imgs[run_num]
    ntrs = raw.shape[-1]
    for tr in range(ntrs):
        print(tr)
        tmp = '/scratch/gpfs/rk1593/rt_mindEye/preprocessing_intermediates/tmp.nii'
        nib.save(index_img(raw,tr),tmp)

        # motion correct and load in the parameters
        mc = '/scratch/gpfs/rk1593/rt_mindEye/preprocessing_intermediates/tmp_mc'
        os.system(f"mcflirt -in {tmp} -reffile /scratch/gpfs/rk1593/rt_mindEye/preprocessing_intermediates/refvol.nii -plots -out {mc}")
        mc_params.append(np.loadtxt(f'{mc}.par'))

        #spatial smoothing
        sm = '/scratch/gpfs/rk1593/rt_mindEye/preprocessing_intermediates/tmp_mc_sm'
        os.system(f'fslmaths {mc} -kernel gauss {5/2.3548} -fmean {sm}')
        imgs.append(get_data(sm + ".nii.gz"))
    
    img = np.rollaxis(np.array(imgs),0,4)
    img = new_img_like(raw,img,copy_header=True)
#     out = f'/scratch/gpfs/rk1593/rt_mindEye/preprocessing_intermediates/sub-01_ses-nsd01_task-nsdcore_run-{run_num:02d}_bold_mc_sm.nii.gz'
#     nib.save(img,out)
    mc_params = np.array(mc_params)
    print(mc_params.shape)
    lsa_glm.fit(run_imgs=img,events=new_events_df, confounds = pd.DataFrame(mc_params))
    fig, ax = plt.subplots(figsize=(10, 10))
    plotting.plot_design_matrix(lsa_glm.design_matrices_[0], ax=ax)
    fig.show()
    trialwise_conditions = new_events_df["trial_type"].unique()
    for condition in trialwise_conditions:
        beta_map = lsa_glm.compute_contrast(str(condition), output_type="effect_size")
        beta_maps_list.append(beta_map.get_fdata().flatten().tolist())
        all_trial_names_list.append(condition)
    print("saving...")
    if mask_on:
        np.save(file = "/scratch/gpfs/rk1593/rt_mindEye/masked_betas_subj1_day1_up_to_run" + str(run_num) + "_" + d4, arr = np.vstack(beta_maps_list))
        np.save(file = "/scratch/gpfs/rk1593/rt_mindEye/masked_trial_ids_up_to_run" + str(run_num) + "_" + d4, arr = np.array(all_trial_names_list))

    else:
        np.save(file = "/scratch/gpfs/rk1593/rt_mindEye/betas_subj1_day1_up_to_run" + str(run_num) + "_" + d4, arr = np.vstack(beta_maps_list))
        np.save(file = "/scratch/gpfs/rk1593/rt_mindEye/trial_ids_up_to_run" + str(run_num) + "_" + d4, arr = np.array(all_trial_names_list))


d4 = Apr-18-2024
0
0


/tmp/ipykernel_485757/2333577167.py:72: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'COCOimage_46003.0_trialNum_1.0_runNum0' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  new_events_df.loc[i_trial, "trial_type"] = "COCOimage_" + str(trial["73k_id"]) + "_trialNum_" + str(trial["trial_number"]) + "_runNum" + str(run_num)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
(188, 6)


/home/rk1593/.conda/envs/rt_mindEye2/lib/python3.11/site-packages/nilearn/glm/first_level/experimental_paradigm.py:167: UserWarning: The following unexpected columns in events data will be ignored: trial_number, response_time, response, missing_data, is_old_session, time, is_correct_session, memory_first, 10k_id, is_correct, total_old_presses, 73k_id, total_novel_presses, change_mind, memory_recent
  warnings.warn(


saving...
1
0


...

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
(188, 6)


/home/rk1593/.conda/envs/rt_mindEye2/lib/python3.11/site-packages/nilearn/glm/first_level/experimental_paradigm.py:167: UserWarning: The following unexpected columns in events data will be ignored: trial_number, response_time, response, missing_data, is_old_session, time, is_correct_session, memory_first, 10k_id, is_correct, total_old_presses, 73k_id, total_novel_presses, change_mind, memory_recent
  warnings.warn(


saving...
3
0


/tmp/ipykernel_485757/2333577167.py:72: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'COCOimage_30374.0_trialNum_1.0_runNum3' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  new_events_df.loc[i_trial, "trial_type"] = "COCOimage_" + str(trial["73k_id"]) + "_trialNum_" + str(trial["trial_number"]) + "_runNum" + str(run_num)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175


In [24]:
pd.DataFrame(mc_params)

,0,1,2,3,4,5
0,0.000000,-0.000219,0.000057,-0.007908,-0.035572,-0.027762
1,0.000380,0.000092,0.000000,0.000002,0.027855,-0.000016
2,-0.000375,-0.000000,-0.000160,-0.000009,0.016802,-0.008750
3,0.000053,-0.000000,-0.000165,0.000010,-0.076325,0.003737
4,-0.000317,-0.000000,-0.000205,-0.005581,0.048545,0.004360
...,...,...,...,...,...,...
183,-0.008011,0.005528,-0.003602,-0.219049,0.211424,0.098236
184,-0.007802,0.005377,-0.003540,-0.213965,0.176155,0.087738
185,-0.008363,0.005457,-0.003659,-0.222344,0.229931,0.085833
186,-0.008037,0.005609,-0.003758,-0.223556,0.180027,0.075353


In [15]:
img = np.rollaxis(np.array(imgs),0,4)
img = new_img_like(raw,img,copy_header=True)
lsa_glm.fit(run_imgs=img,events=new_events_df )

/home/rk1593/.conda/envs/rt_mindEye2/lib/python3.11/site-packages/nilearn/glm/first_level/experimental_paradigm.py:167: UserWarning: The following unexpected columns in events data will be ignored: is_correct_session, total_old_presses, 73k_id, 10k_id, missing_data, total_novel_presses, time, is_correct, change_mind, response_time, memory_first, response, trial_number, is_old_session, memory_recent
  warnings.warn(


FirstLevelModel(drift_model=None, high_pass=None, n_jobs=-1,
                signal_scaling=False, slice_time_ref=0.5, t_r=1.6, verbose=-1)

In [29]:
beta_map.get_fdata().flatten().tolist()


[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0

In [20]:
trialwise_conditions = new_events_df["trial_type"].unique()
for index,condition in enumerate(trialwise_conditions):
    print(index, condition)
    beta_map = lsa_glm.compute_contrast(str(condition), output_type="effect_size")
    beta_maps_list = np.append(beta_maps_list, beta_map.get_fdata().flatten())
    all_trial_names_list = np.append(all_trial_names_list, condition)
np.save(file = "/scratch/gpfs/rk1593/rt_mindEye/betas_subj1_day1_up_to_run" + str(run_num), arr = np.vstack(beta_maps_list))
np.save(file = "/scratch/gpfs/rk1593/rt_mindEye/trial_ids_up_to_run" + str(run_num), arr = np.array(all_trial_names_list))


0 COCOimage_15493.0_trialNum_1.0_runNum4
1 COCOimage_70130.0_trialNum_2.0_runNum4
2 COCOimage_28681.0_trialNum_3.0_runNum4
3 COCOimage_60313.0_trialNum_4.0_runNum4
4 COCOimage_8384.0_trialNum_5.0_runNum4
5 COCOimage_25288.0_trialNum_6.0_runNum4
6 COCOimage_63815.0_trialNum_7.0_runNum4
7 COCOimage_35920.0_trialNum_8.0_runNum4
8 COCOimage_49454.0_trialNum_9.0_runNum4
9 COCOimage_8390.0_trialNum_10.0_runNum4
10 COCOimage_46020.0_trialNum_11.0_runNum4
11 COCOimage_1290.0_trialNum_12.0_runNum4
12 COCOimage_47426.0_trialNum_13.0_runNum4
13 COCOimage_16764.0_trialNum_14.0_runNum4
14 COCOimage_21602.0_trialNum_15.0_runNum4
15 COCOimage_70125.0_trialNum_16.0_runNum4
16 COCOimage_45129.0_trialNum_17.0_runNum4
17 COCOimage_6559.0_trialNum_18.0_runNum4
18 COCOimage_37593.0_trialNum_19.0_runNum4
19 COCOimage_66692.0_trialNum_20.0_runNum4
20 COCOimage_63122.0_trialNum_21.0_runNum4
21 COCOimage_68716.0_trialNum_22.0_runNum4
22 COCOimage_69035.0_trialNum_23.0_runNum4
23 COCOimage_52372.0_trialNum_24.0

KeyboardInterrupt: 

In [ ]:
np.save(file = "/scratch/gpfs/rk1593/rt_mindEye/betas_subj1_day1_up_to_run" + str(run_num), arr = np.vstack(beta_maps_list))

In [ ]:
# Load 73k NSD images
f = h5py.File(f'{data_path}/coco_images_224_float16.hdf5', 'r')
images = f['images'][:] #[:1000] # if you go OOM you can remove the [:] so it isnt preloaded to cpu! (will require a few edits elsewhere tho)
images = torch.Tensor(images).to("cpu").to(data_type)
print("Loaded all 73k possible NSD images to cpu!", images.shape)

In [27]:
beta_maps_list.shape

(487468800,)

## Load models

### CLIP image embeddings  model

In [ ]:
# clip_img_embedder = FrozenOpenCLIPImageEmbedder(
#     arch="ViT-bigG-14",
#     version="laion2b_s39b_b160k",
#     output_tokens=True,
#     only_tokens=True
# )
import pickle
with open("/scratch/gpfs/rk1593/rt_mindEye/clip_img_embedder", "rb") as input_file:
    clip_img_embedder = pickle.load(input_file)
clip_img_embedder.to(device)
clip_seq_dim = 256
clip_emb_dim = 1664

### SD VAE

In [ ]:
if blurry_recon:
    from diffusers import AutoencoderKL    
    autoenc = AutoencoderKL(
        down_block_types=['DownEncoderBlock2D', 'DownEncoderBlock2D', 'DownEncoderBlock2D', 'DownEncoderBlock2D'],
        up_block_types=['UpDecoderBlock2D', 'UpDecoderBlock2D', 'UpDecoderBlock2D', 'UpDecoderBlock2D'],
        block_out_channels=[128, 256, 512, 512],
        layers_per_block=2,
        sample_size=256,
    )
    ckpt = torch.load(f'{cache_dir}/sd_image_var_autoenc.pth')
    autoenc.load_state_dict(ckpt)
    
    autoenc.eval()
    autoenc.requires_grad_(False)
    autoenc.to(device)
    utils.count_params(autoenc)
    
    from autoencoder.convnext import ConvnextXL
    cnx = ConvnextXL(f'{cache_dir}/convnext_xlarge_alpha0.75_fullckpt.pth')
    cnx.requires_grad_(False)
    cnx.eval()
    cnx.to(device)
    
    mean = torch.tensor([0.485, 0.456, 0.406]).to(device).reshape(1,3,1,1)
    std = torch.tensor([0.228, 0.224, 0.225]).to(device).reshape(1,3,1,1)
    
    blur_augs = AugmentationSequential(
        kornia.augmentation.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.2, hue=0.1, p=0.8),
        kornia.augmentation.RandomGrayscale(p=0.1),
        kornia.augmentation.RandomSolarize(p=0.1),
        kornia.augmentation.RandomResizedCrop((224,224), scale=(.9,.9), ratio=(1,1), p=1.0),
        data_keys=["input"],
    )

### MindEye modules

In [ ]:
class MindEyeModule(nn.Module):
    def __init__(self):
        super(MindEyeModule, self).__init__()
    def forward(self, x):
        return x
        
model = MindEyeModule()
model

In [ ]:
class RidgeRegression(torch.nn.Module):
    # make sure to add weight_decay when initializing optimizer
    def __init__(self, input_sizes, out_features, seq_len): 
        super(RidgeRegression, self).__init__()
        self.out_features = out_features
        self.linears = torch.nn.ModuleList([
                torch.nn.Linear(input_size, out_features) for input_size in input_sizes
            ])
    def forward(self, x, subj_idx):
        out = torch.cat([self.linears[subj_idx](x[:,seq]).unsqueeze(1) for seq in range(seq_len)], dim=1)
        return out
        
model.ridge = RidgeRegression(num_voxels_list, out_features=hidden_dim, seq_len=seq_len)
utils.count_params(model.ridge)
utils.count_params(model)

# test on subject 1 with fake data
b = torch.randn((2,seq_len,num_voxels_list[0]))
print(b.shape, model.ridge(b,0).shape)

In [ ]:
from models import BrainNetwork
model.backbone = BrainNetwork(h=hidden_dim, in_dim=hidden_dim, seq_len=seq_len, n_blocks=n_blocks,
                          clip_size=clip_emb_dim, out_dim=clip_emb_dim*clip_seq_dim, 
                          blurry_recon=blurry_recon, clip_scale=clip_scale)
utils.count_params(model.backbone)
utils.count_params(model)

# test that the model works on some fake data
b = torch.randn((2,seq_len,hidden_dim))
print("b.shape",b.shape)

backbone_, clip_, blur_ = model.backbone(b)
print(backbone_.shape, clip_.shape, blur_[0].shape, blur_[1].shape)

### Adding diffusion prior + unCLIP if use_prior=True

In [ ]:
if use_prior:
    from models import *

    # setup diffusion prior network
    out_dim = clip_emb_dim
    depth = 6
    dim_head = 52
    heads = clip_emb_dim//52 # heads * dim_head = clip_emb_dim
    timesteps = 100

    prior_network = PriorNetwork(
            dim=out_dim,
            depth=depth,
            dim_head=dim_head,
            heads=heads,
            causal=False,
            num_tokens = clip_seq_dim,
            learned_query_mode="pos_emb"
        )

    model.diffusion_prior = BrainDiffusionPrior(
        net=prior_network,
        image_embed_dim=out_dim,
        condition_on_text_encodings=False,
        timesteps=timesteps,
        cond_drop_prob=0.2,
        image_embed_scale=None,
    )
    
    utils.count_params(model.diffusion_prior)
    utils.count_params(model)
    
    # prep unCLIP
    if visualize_prior:
        from generative_models.sgm.models.diffusion import DiffusionEngine
        from omegaconf import OmegaConf
        
        config = OmegaConf.load("generative_models/configs/unclip6.yaml")
        config = OmegaConf.to_container(config, resolve=True)
        unclip_params = config["model"]["params"]
        network_config = unclip_params["network_config"]
        denoiser_config = unclip_params["denoiser_config"]
        first_stage_config = unclip_params["first_stage_config"]
        conditioner_config = unclip_params["conditioner_config"]
        sampler_config = unclip_params["sampler_config"]
        scale_factor = unclip_params["scale_factor"]
        disable_first_stage_autocast = unclip_params["disable_first_stage_autocast"]
        offset_noise_level = unclip_params["loss_fn_config"]["params"]["offset_noise_level"]

        first_stage_config['target'] = 'sgm.models.autoencoder.AutoencoderKL'
        sampler_config['params']['num_steps'] = 38

        diffusion_engine = DiffusionEngine(network_config=network_config,
                               denoiser_config=denoiser_config,
                               first_stage_config=first_stage_config,
                               conditioner_config=conditioner_config,
                               sampler_config=sampler_config,
                               scale_factor=scale_factor,
                               disable_first_stage_autocast=disable_first_stage_autocast)
        # set to inference
        diffusion_engine.eval().requires_grad_(False)
        diffusion_engine.to(device)

        ckpt_path = f'{cache_dir}/unclip6_epoch0_step110000.ckpt'
        ckpt = torch.load(ckpt_path, map_location='cpu')
        diffusion_engine.load_state_dict(ckpt['state_dict'])

        image = images[:1].to(device)
        batch={"jpg": image,
              "original_size_as_tuple": torch.ones(image.shape[0], 2).to(device) * 768,
              "crop_coords_top_left": torch.zeros(image.shape[0], 2).to(device)}
        out = diffusion_engine.conditioner(batch)
        vector_suffix = out["vector"].to(device)
        print("vector_suffix", vector_suffix.shape)

### Setup optimizer / lr / ckpt saving

In [ ]:
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']

opt_grouped_parameters = [
    {'params': [p for n, p in model.ridge.named_parameters()], 'weight_decay': 1e-2},
    {'params': [p for n, p in model.backbone.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 1e-2},
    {'params': [p for n, p in model.backbone.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
]
if use_prior:
    opt_grouped_parameters.extend([
        {'params': [p for n, p in model.diffusion_prior.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 1e-2},
        {'params': [p for n, p in model.diffusion_prior.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ])

optimizer = torch.optim.AdamW(opt_grouped_parameters, lr=max_lr)

if lr_scheduler_type == 'linear':
    lr_scheduler = torch.optim.lr_scheduler.LinearLR(
        optimizer,
        total_iters=int(np.floor(num_epochs*num_iterations_per_epoch)),
        last_epoch=-1
    )
elif lr_scheduler_type == 'cycle':
    total_steps=int(np.floor(num_epochs*num_iterations_per_epoch))
    print("total_steps", total_steps)
    lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer, 
        max_lr=max_lr,
        total_steps=total_steps,
        final_div_factor=1000,
        last_epoch=-1, pct_start=2/num_epochs
    )
    
def save_ckpt(tag):
    ckpt_path = outdir + "/" + tag
    if accelerator.is_main_process:
        unwrapped_model = accelerator.unwrap_model(model)
        torch.save({
            'epoch': epoch,
            'model_state_dict': unwrapped_model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'lr_scheduler': lr_scheduler.state_dict(),
            'train_losses': losses,
            'test_losses': test_losses,
            'lrs': lrs,
            }, ckpt_path)
    print(f"\n---saved {outdir}/{tag} ckpt!---\n")

def load_ckpt(tag,load_lr=True,load_optimizer=True,load_epoch=True,strict=True,outdir=outdir,multisubj_loading=False): 
    print(f"\n---loading {outdir}/{tag}.pth ckpt---\n")
    checkpoint = torch.load(outdir+'/last.pth', map_location='cpu')
    state_dict = checkpoint['model_state_dict']
    if multisubj_loading: # remove incompatible ridge layer that will otherwise error
        state_dict.pop('ridge.linears.0.weight',None)
    model.load_state_dict(state_dict, strict=strict)
    if load_epoch:
        globals()["epoch"] = checkpoint['epoch']
        print("Epoch",epoch)
    if load_optimizer:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    if load_lr:
        lr_scheduler.load_state_dict(checkpoint['lr_scheduler'])
    del checkpoint
        
print("\nDone with model preparations!")
num_params = utils.count_params(model)

# Weights and Biases

In [ ]:
wandb_log = False
if local_rank==0 and wandb_log: # only use main process for wandb logging
    import wandb
    wandb_project = 'mindeye'
    print(f"wandb {wandb_project} run {model_name}")
    # need to configure wandb beforehand in terminal with "wandb init"!
    wandb_config = {
      "model_name": model_name,
      "global_batch_size": global_batch_size,
      "batch_size": batch_size,
      "num_epochs": num_epochs,
      "num_sessions": num_sessions,
      "num_params": num_params,
      "clip_scale": clip_scale,
      "prior_scale": prior_scale,
      "blur_scale": blur_scale,
      "use_image_aug": use_image_aug,
      "max_lr": max_lr,
      "mixup_pct": mixup_pct,
      "num_samples_per_epoch": num_samples_per_epoch,
      "num_test": num_test,
      "ckpt_interval": ckpt_interval,
      "ckpt_saving": ckpt_saving,
      "seed": seed,
      "distributed": distributed,
      "num_devices": num_devices,
      "world_size": world_size,
      "train_url": train_url,
      "test_url": test_url,
    }
    print("wandb_config:\n",wandb_config)
    print("wandb_id:",model_name)
    wandb.init(
        project=wandb_project,
        name=model_name,
        config=wandb_config,
        resume="allow",
    )
else:
    wandb_log = False

# Main

In [ ]:
epoch = 100
losses, test_losses, lrs = [], [], []
best_test_loss = 1e9
torch.cuda.empty_cache()

In [ ]:
multisubject_ckpt

In [ ]:
'/scratch/gpfs/rk1593/rt_mindEye/multisubject_subj01_1024hid_nolow_300ep'

In [ ]:
# load multisubject stage1 ckpt if set
if multisubject_ckpt is not None and not resume_from_ckpt:
    load_ckpt("100_epochs_fine_tune_subj1day1Mar-21-2024.pth",outdir=multisubject_ckpt,load_lr=False,load_optimizer=False,load_epoch=False,strict=False,multisubj_loading=True)

In [ ]:
# load saved ckpt model weights into current model
if resume_from_ckpt:
    load_ckpt("last",load_lr=True,load_optimizer=True,load_epoch=True)
elif wandb_log:
    if wandb.run.resumed:
        load_ckpt("last",load_lr=True,load_optimizer=True,load_epoch=True)

In [ ]:
train_dls = [train_dl[f'subj0{s}'] for s in subj_list]

model, optimizer, *train_dls, lr_scheduler = accelerator.prepare(model, optimizer, *train_dls, lr_scheduler)
# leaving out test_dl since we will only have local_rank 0 device do evals

In [ ]:
for s, train_dl in enumerate(train_dls):
    print(s)
    with torch.cuda.amp.autocast(dtype=data_type):
        for iter, (behav0, past_behav0, future_behav0, old_behav0) in enumerate(train_dl):  
             pass
           # print(iter, behav0.shape)

In [ ]:
behav0.shape

In [ ]:
print(f"{model_name} starting with epoch {epoch} / {num_epochs}")
progress_bar = tqdm(range(epoch,num_epochs), ncols=1200, disable=(local_rank!=0))
test_image, test_voxel = None, None
mse = nn.MSELoss()
l1 = nn.L1Loss()
soft_loss_temps = utils.cosine_anneal(0.004, 0.0075, num_epochs - int(mixup_pct * num_epochs))
from datetime import date
today = date.today()
datetime_today4 = today.strftime("%b-%d-%Y")

for epoch in progress_bar:
    model.train()

    fwd_percent_correct = 0.
    bwd_percent_correct = 0.
    test_fwd_percent_correct = 0.
    test_bwd_percent_correct = 0.
    
    recon_cossim = 0.
    test_recon_cossim = 0.
    recon_mse = 0.
    test_recon_mse = 0.

    loss_clip_total = 0.
    loss_blurry_total = 0.
    loss_blurry_cont_total = 0.
    test_loss_clip_total = 0.
    
    loss_prior_total = 0.
    test_loss_prior_total = 0.

    blurry_pixcorr = 0.
    test_blurry_pixcorr = 0. # needs >.456 to beat low-level subj01 results in mindeye v1

    # pre-load all batches for this epoch (it's MUCH faster to pre-load in bulk than to separate loading per batch)
    voxel_iters = {} # empty dict because diff subjects have differing # of voxels
    image_iters = torch.zeros(num_iterations_per_epoch, batch_size*len(subj_list), 3, 224, 224).float()
    annot_iters = {}
    perm_iters, betas_iters, select_iters = {}, {}, {}
    for s, train_dl in enumerate(train_dls):
        with torch.cuda.amp.autocast(dtype=data_type):
            for iter, (behav0, past_behav0, future_behav0, old_behav0) in enumerate(train_dl):  
          
                image0 = images[behav0[:,0,0].cpu().long()].float()
                image_iters[iter,s*batch_size:s*batch_size+batch_size] = image0
                print("behav0[:,0,5].cpu().long(): ", behav0[:,0,5].cpu().long())
                voxel0 = voxels[f'subj0{subj_list[s]}'][behav0[:,0,5].cpu().long()]
                # ipdb> voxel0.shape => torch.Size([64, 15724]), get this 64 trials out of the 30K with the beta matrix for those trials
                # so we would want to feed in each run as training data
                # we wouold want to read in all the runs from day 1 and create a np.zeros(30K,15724)matrix but then
                # populate the day 1 trials with the correct beta matrices
                # 
                import pdb 
                pdb.set_trace()
                voxel0 = torch.Tensor(voxel0)
                voxel0 = voxel0.unsqueeze(1)

                if epoch < int(mixup_pct * num_epochs):
                    voxel0, perm, betas, select = utils.mixco(voxel0)
                    perm_iters[f"subj0{subj_list[s]}_iter{iter}"] = perm
                    betas_iters[f"subj0{subj_list[s]}_iter{iter}"] = betas
                    select_iters[f"subj0{subj_list[s]}_iter{iter}"] = select

                voxel_iters[f"subj0{subj_list[s]}_iter{iter}"] = voxel0

                if iter >= num_iterations_per_epoch-1:
                    break

    # you now have voxel_iters and image_iters with num_iterations_per_epoch batches each
    for train_i in range(num_iterations_per_epoch):
        with torch.cuda.amp.autocast(dtype=data_type):
            optimizer.zero_grad()
            loss=0.

            voxel_list = [voxel_iters[f"subj0{s}_iter{train_i}"].detach().to(device) for s in subj_list]
            image = image_iters[train_i].detach()
            image = image.to(device)

            if use_image_aug: 
                image = img_augment(image)

            clip_target = clip_img_embedder(image)
            assert not torch.any(torch.isnan(clip_target))

            if epoch < int(mixup_pct * num_epochs):
                perm_list = [perm_iters[f"subj0{s}_iter{train_i}"].detach().to(device) for s in subj_list]
                perm = torch.cat(perm_list, dim=0)
                betas_list = [betas_iters[f"subj0{s}_iter{train_i}"].detach().to(device) for s in subj_list]
                betas = torch.cat(betas_list, dim=0)
                select_list = [select_iters[f"subj0{s}_iter{train_i}"].detach().to(device) for s in subj_list]
                select = torch.cat(select_list, dim=0)

            voxel_ridge_list = [model.ridge(voxel_list[si],si) for si,s in enumerate(subj_list)]
            voxel_ridge = torch.cat(voxel_ridge_list, dim=0)

            backbone, clip_voxels, blurry_image_enc_ = model.backbone(voxel_ridge)

            if clip_scale>0:
                clip_voxels_norm = nn.functional.normalize(clip_voxels.flatten(1), dim=-1)
                clip_target_norm = nn.functional.normalize(clip_target.flatten(1), dim=-1)

            if use_prior:
                loss_prior, prior_out = model.diffusion_prior(text_embed=backbone, image_embed=clip_target)
                loss_prior_total += loss_prior.item()
                loss_prior *= prior_scale
                loss += loss_prior

                recon_cossim += nn.functional.cosine_similarity(prior_out, clip_target).mean().item()
                recon_mse += mse(prior_out, clip_target).item()

            if clip_scale>0:
                if epoch < int(mixup_pct * num_epochs):                
                    loss_clip = utils.mixco_nce(
                        clip_voxels_norm,
                        clip_target_norm,
                        temp=.006,
                        perm=perm, betas=betas, select=select)
                else:
                    epoch_temp = soft_loss_temps[epoch-int(mixup_pct*num_epochs)]
                    loss_clip = utils.soft_clip_loss(
                        clip_voxels_norm,
                        clip_target_norm,
                        temp=epoch_temp)

                loss_clip_total += loss_clip.item()
                loss_clip *= clip_scale
                loss += loss_clip

            if blurry_recon:     
                image_enc_pred, transformer_feats = blurry_image_enc_

                image_enc = autoenc.encode(2*image-1).latent_dist.mode() * 0.18215
                loss_blurry = l1(image_enc_pred, image_enc)
                loss_blurry_total += loss_blurry.item()

                if epoch < int(mixup_pct * num_epochs):
                    image_enc_shuf = image_enc[perm]
                    betas_shape = [-1] + [1]*(len(image_enc.shape)-1)
                    image_enc[select] = image_enc[select] * betas[select].reshape(*betas_shape) + \
                        image_enc_shuf[select] * (1 - betas[select]).reshape(*betas_shape)

                image_norm = (image - mean)/std
                image_aug = (blur_augs(image) - mean)/std
                _, cnx_embeds = cnx(image_norm)
                _, cnx_aug_embeds = cnx(image_aug)

                cont_loss = utils.soft_cont_loss(
                    nn.functional.normalize(transformer_feats.reshape(-1, transformer_feats.shape[-1]), dim=-1),
                    nn.functional.normalize(cnx_embeds.reshape(-1, cnx_embeds.shape[-1]), dim=-1),
                    nn.functional.normalize(cnx_aug_embeds.reshape(-1, cnx_embeds.shape[-1]), dim=-1),
                    temp=0.2)
                loss_blurry_cont_total += cont_loss.item()

                loss += (loss_blurry + 0.1*cont_loss) * blur_scale #/.18215

            if clip_scale>0:
                # forward and backward top 1 accuracy        
                labels = torch.arange(len(clip_voxels_norm)).to(clip_voxels_norm.device) 
                fwd_percent_correct += utils.topk(utils.batchwise_cosine_similarity(clip_voxels_norm, clip_target_norm), labels, k=1).item()
                bwd_percent_correct += utils.topk(utils.batchwise_cosine_similarity(clip_target_norm, clip_voxels_norm), labels, k=1).item()

            if blurry_recon:
                with torch.no_grad():
                    # only doing pixcorr eval on a subset of the samples per batch because its costly & slow to compute autoenc.decode()
                    random_samps = np.random.choice(np.arange(len(image)), size=len(image)//5, replace=False)
                    blurry_recon_images = (autoenc.decode(image_enc_pred[random_samps]/0.18215).sample/ 2 + 0.5).clamp(0,1)
                    pixcorr = utils.pixcorr(image[random_samps], blurry_recon_images)
                    blurry_pixcorr += pixcorr.item()

            utils.check_loss(loss)
            accelerator.backward(loss)
            optimizer.step()

            losses.append(loss.item())
            lrs.append(optimizer.param_groups[0]['lr'])

            if lr_scheduler_type is not None:
                lr_scheduler.step()


            # if finished training, save jpg recons if they exist
            if (epoch == num_epochs-1) or (epoch % ckpt_interval == 0):
                if blurry_recon:    
                    image_enc = autoenc.encode(2*image[:4]-1).latent_dist.mode() * 0.18215
                    # transform blurry recon latents to images and plot it
                    fig, axes = plt.subplots(1, 8, figsize=(10, 4))
                    jj=-1
                    for j in [0,1,2,3]:
                        jj+=1
                        axes[jj].imshow(utils.torch_to_Image((autoenc.decode(image_enc[[j]]/0.18215).sample / 2 + 0.5).clamp(0,1)))
                        axes[jj].axis('off')
                        jj+=1
                        axes[jj].imshow(utils.torch_to_Image((autoenc.decode(image_enc_pred[[j]]/0.18215).sample / 2 + 0.5).clamp(0,1)))
                        axes[jj].axis('off')

                    if wandb_log:
                        logs[f"test/blur_recons"] = wandb.Image(fig, caption=f"epoch{epoch:03d}")
                        plt.close()
                    else:
                        plt.show()
                        
                if use_prior and visualize_prior: # output recons every ckpt
                    idx = np.random.randint(0, 3)
                    print(f"reconstructing... idx={idx}")
                    samples = utils.unclip_recon(prior_out[[idx]],
                             diffusion_engine,
                             vector_suffix)
                    if wandb_log:
                        logs[f"test/orig"] = wandb.Image(transforms.ToPILImage()(image[idx]),
                                                           caption=f"epoch{epoch:03d}")
                        logs[f"test/recons"] = wandb.Image(transforms.ToPILImage()(samples[0]),
                                                           caption=f"epoch{epoch:03d}")
                    if utils.is_interactive():
                        plt.figure(figsize=(2,2))
                        plt.imshow(transforms.ToPILImage()(image[idx]))
                        plt.axis('off')
                        plt.show()
                        
                        plt.figure(figsize=(2,2))
                        plt.imshow(transforms.ToPILImage()(samples[0]))
                        plt.axis('off')
                        plt.show()

          

            
    # Save model checkpoint and reconstruct
    if (ckpt_saving) and (epoch % ckpt_interval == 0):
         save_ckpt(str(epoch) + "_epochs_fine_tune_subj1day1" + datetime_today4 + ".pth")

    # wait for other GPUs to catch up if needed
    accelerator.wait_for_everyone()
    torch.cuda.empty_cache()
    gc.collect()

print("\n===Finished!===\n")
if ckpt_saving:
    save_ckpt(str(epoch) +"_epochs_fine_tune_subj1day1" + datetime_today4 + ".pth")

In [ ]:
plt.plot(losses)
plt.show()
plt.plot(test_losses)
plt.show()